# Przetwarzanie Danych w językach R i Python Projekt 2
# Piotr Rowicki 320730


## Wstęp 
Podobnie jak język R, pyhon również udostępnia narzędzie do przetwarzania danych. Jest nim biblioteka pandas. Podobnie jak w projekcie 1, porównamy jej działanie z zapytaniami SQL.
Zapytania mają dokładnie taką samą tresć jak te z pierwszego projektu, więc interpretacja ich zostanie pominięta. A ponieważ implementowane będą one tylko jedną petodą (poza wzorcową, która przez swój charakter nie ma szans być wydajniejsza), porównania czasowe również zostaną pominięte 

In [73]:
#załączenie dependencji
from  Rowicki_Piotr_PD2 import *
import pandas as pd
import sqlite3
import os

In [74]:
# wczytanie danych
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv.gz",compression = 'gzip')
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv.gz",compression = 'gzip')
Users = pd.read_csv("travel_stackexchange_com/Users.csv.gz",compression = 'gzip')
PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv.gz",compression = 'gzip')

In [75]:
# ustanowienie połączenia i wypełnienie bazy danych
database=os.path.join("stackexchange.db")
if os.path.exists(database):
    os.remove(database)
conn = sqlite3.connect(database)
PostLinks.to_sql("PostLinks",conn)
Posts.to_sql("Posts",conn)
Comments.to_sql("Comments",conn)
Users.to_sql("Users",conn)

95922

In [76]:
# w przypadku kiedy baza juz istnieje
database=os.path.join("stackexchange.db")
conn = sqlite3.connect(database)

In [77]:
# pomocnicza funkcja do porównywania ramek
def compare(x,y):
    if x.shape[0]!=y.shape[0] or x.shape[1]!=y.shape[1]:
        return False
    return x.sort_values(by=x.columns.tolist(), ascending=False,ignore_index=True).equals(y.sort_values(by=y.columns.tolist(), ascending=False,ignore_index=True))

## Zapytanie 1


### SQL

In [78]:
sqlRes_1=pd.read_sql_query(sql="""SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10
""",con=conn)

In [79]:
#Wynik zapytania
sqlRes_1

,Location,Count
0,"Christchurch, New Zealand",2795
1,"New York, NY",1872
2,"London, United Kingdom",1711
3,UK,1707
4,"Sunshine Coast QLD, Australia",1558
5,Australia,1199
6,Seat 21A,1056
7,"Vancouver, Canada",1051
8,Netherlands,1013
9,"London, UK",947


### Pandas

In [80]:
pdRes_1=solution_1(Posts,Users)

In [81]:
#wynik
pdRes_1

,Location,Count
0,"Christchurch, New Zealand",2795
1,"New York, NY",1872
2,"London, United Kingdom",1711
3,UK,1707
4,"Sunshine Coast QLD, Australia",1558
5,Australia,1199
6,Seat 21A,1056
7,"Vancouver, Canada",1051
8,Netherlands,1013
9,"London, UK",947


### porównanie zgodnosci

In [82]:
pdRes_1.equals(sqlRes_1)

True

## zapytanie 2


### SQL

In [83]:
#wewnetrzna kwerenda, w celu obejscia problemow z biblioteka
x=pd.read_sql(sql="""SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
    FROM PostLinks
    GROUP BY RelatedPostId""",con=conn)


In [84]:
# jednorazowe dopisanie nowej tabeli do bazy danych
x.to_sql("RelatedTab",conn,index=False)

8369

In [85]:
sqlRes_2=pd.read_sql_query(sql="""SELECT Posts.Title, RelatedTab.NumLinks
    FROM
     RelatedTab
    JOIN Posts ON RelatedTab.PostId=Posts.Id
    WHERE Posts.PostTypeId=1
    ORDER BY NumLinks DESC
 """,con=conn)
sqlRes_2=sqlRes_2

In [86]:
#wynik
sqlRes_2

,Title,NumLinks
0,Is there a way to find out if I need a transit...,1157
1,Do I need a visa to transit (or layover) in th...,1012
2,Should I submit bank statements when applying ...,669
3,UK visa refusal on V 4.2 a + c (and sometimes ...,523
4,Should my first trip be to the country which i...,461
...,...,...
8094,Why can't I reserve a seat on the Eurocity tra...,1
8095,Independent Research in the US under B1 visa,1
8096,2 round tickets plus 1 back ticket to Thailand,1
8097,Which flight search engines show the flight op...,1


### Pandas

In [87]:
pdRes_2=solution_2(Posts,PostLinks)
pdRes_2

,Title,NumLinks
0,Is there a way to find out if I need a transit...,1157
1,Do I need a visa to transit (or layover) in th...,1012
2,Should I submit bank statements when applying ...,669
3,UK visa refusal on V 4.2 a + c (and sometimes ...,523
4,Should my first trip be to the country which i...,461
...,...,...
8094,"Do I have to go through customs, recheck my ba...",1
8095,Bringing tea bags in zip lock bag to Korea?,1
8096,Renting a flat in Lille for a few months - opt...,1
8097,Alternative to public hot water bath in Japan ...,1


### porównanie zgodności

In [88]:
compare(pdRes_2,sqlRes_2)

True

## Zapytanie 3

### SQL

In [89]:
sqlRes_3=pd.read_sql_query(sql="""SELECT Title, CommentCount, ViewCount, CommentsTotalScore,
DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
"""
,con=conn)
# W celu usprawnienia wywołania, limit 10 na końcu zapytania zostało pominięte. Z tego powodu z wyniku wybieramy tylko pierwsze 10 wierszy
sqlRes_3=sqlRes_3[0:10]

In [90]:
# wynik
sqlRes_3

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,61309.0,630,Megha,1679,None
1,How to intentionally get denied entry to the U...,37,72856.0,618,user11743,1997,None
2,Can I wear a bulletproof vest while traveling ...,26,23467.0,403,Ulkoma,9190,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,113001.0,388,hippietrail,75994,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,14308.0,358,Emma-louise,611,None
5,Immigration officer that stopped me at the air...,24,22495.0,345,DiegoJ,1273,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,29344.0,343,Silver Dragon,943,"San Francisco, United States"
7,Can I fly with a gold bar?,25,55450.0,333,Ulkoma,9190,"London, United Kingdom"
8,How to book a flight if my passport doesn't st...,23,18029.0,330,kukis,894,"Sztokholm, Szwecja"
9,Why don't airlines have backup planes just in ...,26,18809.0,328,clickbait,542,None


### wynik implementacji w Pandas

In [91]:
pdRes_3=solution_3(Comments,Posts,Users)

In [92]:
# wynik
pdRes_3

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,61309.0,630,Megha,1679,NaN
1,How to intentionally get denied entry to the U...,37,72856.0,618,user11743,1997,NaN
2,Can I wear a bulletproof vest while traveling ...,26,23467.0,403,Ulkoma,9190,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,113001.0,388,hippietrail,75994,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,14308.0,358,Emma-louise,611,NaN
5,Immigration officer that stopped me at the air...,24,22495.0,345,DiegoJ,1273,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,29344.0,343,Silver Dragon,943,"San Francisco, United States"
7,Can I fly with a gold bar?,25,55450.0,333,Ulkoma,9190,"London, United Kingdom"
8,How to book a flight if my passport doesn't st...,23,18029.0,330,kukis,894,"Sztokholm, Szwecja"
9,Why don't airlines have backup planes just in ...,26,18809.0,328,clickbait,542,NaN


### porównanie wyników

In [93]:
pdRes_3.equals(sqlRes_3)

True

## Zapytanie 4

### SQL

In [94]:
sqlRes_4=pd.read_sql_query(sql="""SELECT DisplayName, QuestionsNumber, AnswersNumber, Location,
Reputation, UpVotes, DownVotes
FROM (
SELECT *
FROM (
SELECT COUNT(*) as AnswersNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 2
GROUP BY OwnerUserId
) AS Answers
JOIN
(
SELECT COUNT(*) as QuestionsNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
ON Answers.OwnerUserId = Questions.OwnerUserId
WHERE AnswersNumber > QuestionsNumber
ORDER BY AnswersNumber DESC
LIMIT 5
) AS PostsCounts
JOIN Users ON PostsCounts.OwnerUserId = Users.Id""",
con=conn)

In [95]:
sqlRes_4

,DisplayName,QuestionsNumber,AnswersNumber,Location,Reputation,UpVotes,DownVotes
0,Mark Mayo,344,1968,"Christchurch, New Zealand",157193,16082,1661
1,phoog,12,1690,"New York, NY",120317,8767,567
2,Relaxed,8,1506,None,99185,11268,606
3,lambshaanxy,117,1466,None,98555,15021,275
4,Crazydre,172,1163,None,73180,2263,132


### Pandas

In [96]:
pdRes_4=solution_4(Posts,Users)


In [97]:
#wynik
pdRes_4

,DisplayName,QuestionsNumber,AnswersNumber,Location,Reputation,UpVotes,DownVotes
0,Mark Mayo,344,1968,"Christchurch, New Zealand",157193,16082,1661
1,phoog,12,1690,"New York, NY",120317,8767,567
2,Relaxed,8,1506,NaN,99185,11268,606
3,lambshaanxy,117,1466,NaN,98555,15021,275
4,Crazydre,172,1163,NaN,73180,2263,132


### porównianie wyników

In [98]:
pdRes_4.equals(sqlRes_4)

True

## Zapytanie 5

### SQL

In [99]:
sqlRes_5=pd.read_sql_query(sql="""SELECT
Users.AccountId,
Users.DisplayName,
Users.Location,
AVG(PostAuth.AnswersCount) as AverageAnswersCount
FROM
(
SELECT
AnsCount.AnswersCount,
Posts.Id,
Posts.OwnerUserId
FROM (
SELECT Posts.ParentId, COUNT(*) AS AnswersCount
FROM Posts
WHERE Posts.PostTypeId = 2
GROUP BY Posts.ParentId
) AS AnsCount
JOIN Posts ON Posts.Id = AnsCount.ParentId
) AS PostAuth
JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
GROUP BY OwnerUserId
ORDER BY AverageAnswersCount DESC
LIMIT 10
""",con=conn)

In [100]:
sqlRes_5

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,79346.0,Thomas Matthews,California,8.0
6,54571.0,Christian,None,8.0
7,42364.0,Petrogad,None,8.0
8,20473.0,Jeremy Boyd,"Houston, TX",8.0
9,19588.0,JD Isaacks,"Atlanta, GA",8.0


### Pandas

In [101]:
pdRes_5=solution_5(Posts, Users)

In [102]:
#wynik
pdRes_5

,AccountId,DisplayName,Location,AverageAnswersCount
0,40811.0,vocaro,"San Jose, CA",11.0
1,280.0,csmba,"San Francisco, CA",11.0
2,44093.0,Emma Arbogast,"Salem, OR",10.0
3,204.0,Josh,Australia,10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,19588.0,JD Isaacks,"Atlanta, GA",8.0
6,54571.0,Christian,NaN,8.0
7,42364.0,Petrogad,NaN,8.0
8,79346.0,Thomas Matthews,California,8.0
9,20473.0,Jeremy Boyd,"Houston, TX",8.0


### porówanie wyników

In [103]:
compare(pdRes_5,sqlRes_5)

True

## Podsumowanie
Ponownie udało się odtowrzyć zadane zapytania korzystając z funkcjonalnosci biblioteki pandas.
Porównując z wczesniej uzytych bibliotek, byla ona najprzyjemniejsza do użytku. Podobnie jak dplyr, składania była podobna do narzędzi obecnych w innych językach( LINQu w C#,  streams w Javie), przez co jej użycie było bardzo naturalne.Ponadto, same funkcjonalnosci jakie zapewnie język, były o wiele przyjemniejsze do stosowania.

In [104]:
#zamknięcie połączenia z bazą danych
conn.close()